[![GitHub](https://img.shields.io/badge/Github-hibana2077-blue?style=plastic-square&logo=github)](https://github.com/hibana2077)
[![Colab](https://img.shields.io/badge/Colab-Open%20in%20Colab-blue?style=plastic-square&logo=googlecolab)](https://colab.research.google.com/github/hibana2077/hibana2077/blob/master/train/train.ipynb)

如果要訓練這份資料集會需要安裝talib套件，請參考[這裡](https://www.lfd.uci.edu/~gohlke/pythonlibs/#ta-lib)下載對應的版本，並使用pip安裝。

In [1]:
from ccxt import binance
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
from torch.nn import functional as F
from talib import abstract
import pandas as pd
import numpy as np
import torch
import os
import sys

# 環境整理

In [2]:
ls_dir = os.listdir(path="..")
if "data" not in ls_dir:
    os.mkdir(path="../data")
if "model" not in ls_dir:
    os.mkdir(path="../model")
if "data" in ls_dir:
    ls_dir = os.listdir(path="../data")
    #remove all files in data folder
    for file in ls_dir:os.remove(path="../data/"+file)

# 下載資料集

In [3]:
# Binance BTC/USDT 1h candles from 2020-01-01 to 2021-01-01

binance = binance()
symbol = 'BTC/USDT'
timeframe = '1h'
file_name = f"../data/{symbol.replace('/', '_')}_{timeframe}.csv"
start = binance.parse8601('2020-01-01T00:00:00Z')
end = binance.parse8601('2022-01-01T00:00:00Z')
cnt_time = start
data = []
while cnt_time < end:
    ohlcv = binance.fetch_ohlcv(symbol, timeframe, cnt_time)
    data += ohlcv
    cnt_time = ohlcv[-1][0] + 3600000 # 1h in ms    
df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
df['time'] = pd.to_datetime(df['time'], unit='ms')
df.to_csv(file_name, index=False)

# 讀取資料集

In [4]:
#如果有下载好的數據，可以直接讀取
data_file = '../data/btc_usdt_1h.csv' #-> 可以自行更換
df = pd.read_csv(data_file)

# 數據處理

- 計算RSI
- 計算MACD
- 計算OBV
- 計算CCI
- 改成變化百分比 -> 標準化

關於技術指標的說明可以參考[這裡](https://www.investopedia.com/terms/t/technicalindicator.asp)，或是google。

In [5]:
df['RSI'] = abstract.RSI(df, timeperiod=14)
df['MACD'] = abstract.MACD(df, fastperiod=12, slowperiod=26, signalperiod=9)['macd'] #只取MACD
df['OBV'] = abstract.OBV(df, timeperiod=14)
df['CCI'] = abstract.CCI(df, timeperiod=14)
df['OPEN_percent'] = df['open'].pct_change()
df['CLOSE_percent'] = df['close'].pct_change()
df['HIGH_percent'] = df['high'].pct_change()
df['LOW_percent'] = df['low'].pct_change()
df['VOLUME_percent'] = df['volume']*df['close'].pct_change()
#由於RSI MACD OBV CCI 他們已經是標準化的，所以不需要再標準化

# 設定買賣點

將買賣點分為下跌、上漲、不動，並將數據轉成one-hot編碼。

In [6]:
df['UP'] = df['CLOSE_percent'].apply(lambda x: 1 if x > 0 else 0)
df['DOWN'] = df['CLOSE_percent'].apply(lambda x: 1 if x < 0 else 0)
df['UP'] = df['UP'].shift(-1) #shift UP DOWN 一個單位，因為我們要預測的是下一個時間點的漲跌
df['DOWN'] = df['DOWN'].shift(-1)

df = df.dropna()


In [7]:
df

,time,open,high,low,close,volume,RSI,MACD,OBV,CCI,OPEN_percent,CLOSE_percent,HIGH_percent,LOW_percent,VOLUME_percent,UP,DOWN
33,2020-01-02 09:00:00,7153.57,7165.00,7135.36,7162.01,840.001328,46.653946,-18.772873,-1085.293960,-35.190579,0.005989,0.001069,0.001051,0.003692,0.898195,0.0,1.0
34,2020-01-02 10:00:00,7162.18,7180.00,7153.33,7161.83,1446.219984,46.612835,-17.585034,-2531.513944,-4.322773,0.001204,-0.000025,0.002094,0.002518,-0.036347,0.0,1.0
35,2020-01-02 11:00:00,7161.89,7168.67,7139.03,7139.79,761.156570,41.760421,-18.212168,-3292.670514,-32.074422,-0.000040,-0.003077,-0.001578,-0.001999,-2.342403,1.0,0.0
36,2020-01-02 12:00:00,7139.73,7163.40,7139.03,7158.29,794.030497,46.769498,-17.020182,-2498.640017,-11.448699,-0.003094,0.002591,-0.000735,0.000000,2.057422,0.0,1.0
37,2020-01-02 13:00:00,7158.86,7163.35,7107.43,7131.15,1566.280693,41.174606,-18.057343,-4064.920710,-69.799501,0.002679,-0.003791,-0.000007,-0.004426,-5.938410,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17994,2022-01-21 01:00:00,40894.81,40929.76,39730.00,39762.99,4172.351850,25.308075,-334.514225,697285.662621,-152.190082,0.005258,-0.027676,-0.004142,-0.016682,-115.474572,1.0,0.0
17995,2022-01-21 02:00:00,39763.00,40280.00,39267.86,39830.52,4354.458167,26.469820,-445.353256,701640.120788,-137.983834,-0.027676,0.001698,-0.015875,-0.011632,7.395233,0.0,1.0
17996,2022-01-21 03:00:00,39835.99,39944.45,38220.00,38465.65,9508.823455,19.775068,-635.996220,692131.297333,-150.716081,0.001836,-0.034267,-0.008330,-0.026685,-325.838273,1.0,0.0
17997,2022-01-21 04:00:00,38465.65,39070.00,38437.15,38842.03,3091.420650,25.379860,-748.087930,695222.717983,-126.902770,-0.034400,0.009785,-0.021892,0.005682,30.249038,1.0,0.0


In [8]:
#正規化
df['RSI'] = (df['RSI'] - df['RSI'].min()) / (df['RSI'].max() - df['RSI'].min())
df['MACD'] = (df['MACD'] - df['MACD'].min()) / (df['MACD'].max() - df['MACD'].min())
df['OBV'] = (df['OBV'] - df['OBV'].min()) / (df['OBV'].max() - df['OBV'].min())
df['CCI'] = (df['CCI'] - df['CCI'].min()) / (df['CCI'].max() - df['CCI'].min())
df['OPEN_percent'] = (df['OPEN_percent'] - df['OPEN_percent'].min()) / (df['OPEN_percent'].max() - df['OPEN_percent'].min())
df['CLOSE_percent'] = (df['CLOSE_percent'] - df['CLOSE_percent'].min()) / (df['CLOSE_percent'].max() - df['CLOSE_percent'].min())
df['HIGH_percent'] = (df['HIGH_percent'] - df['HIGH_percent'].min()) / (df['HIGH_percent'].max() - df['HIGH_percent'].min())
df['LOW_percent'] = (df['LOW_percent'] - df['LOW_percent'].min()) / (df['LOW_percent'].max() - df['LOW_percent'].min())
df['VOLUME_percent'] = (df['VOLUME_percent'] - df['VOLUME_percent'].min()) / (df['VOLUME_percent'].max() - df['VOLUME_percent'].min())

C:\Users\李軒豪\AppData\Local\Temp\ipykernel_34144\3759880002.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RSI'] = (df['RSI'] - df['RSI'].min()) / (df['RSI'].max() - df['RSI'].min())
C:\Users\李軒豪\AppData\Local\Temp\ipykernel_34144\3759880002.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD'] = (df['MACD'] - df['MACD'].min()) / (df['MACD'].max() - df['MACD'].min())
C:\Users\李軒豪\AppData\Local\Temp\ipykernel_34144\3759880002.py:4: SettingWithCopyWarning: 
A value is trying to be set on a co

In [9]:
df

,time,open,high,low,close,volume,RSI,MACD,OBV,CCI,OPEN_percent,CLOSE_percent,HIGH_percent,LOW_percent,VOLUME_percent,UP,DOWN
33,2020-01-02 09:00:00,7153.57,7165.00,7135.36,7162.01,840.001328,0.439139,0.551337,0.126083,0.462566,0.528817,0.514627,0.520187,0.480495,0.489460,0.0,1.0
34,2020-01-02 10:00:00,7162.18,7180.00,7153.33,7161.83,1446.219984,0.438670,0.551617,0.124966,0.495765,0.515369,0.511553,0.524332,0.478217,0.489393,0.0,1.0
35,2020-01-02 11:00:00,7161.89,7168.67,7139.03,7139.79,761.156570,0.383355,0.551469,0.124377,0.465917,0.511873,0.502978,0.509739,0.469449,0.489230,1.0,0.0
36,2020-01-02 12:00:00,7139.73,7163.40,7139.03,7158.29,794.030497,0.440456,0.551750,0.124991,0.488101,0.503292,0.518903,0.513089,0.473329,0.489542,0.0,1.0
37,2020-01-02 13:00:00,7158.86,7163.35,7107.43,7131.15,1566.280693,0.376677,0.551505,0.123781,0.425343,0.519516,0.500972,0.515984,0.464738,0.488975,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17994,2022-01-21 01:00:00,40894.81,40929.76,39730.00,39762.99,4172.351850,0.195806,0.477013,0.665788,0.336728,0.526761,0.433872,0.499548,0.440951,0.481210,1.0,0.0
17995,2022-01-21 02:00:00,39763.00,40280.00,39267.86,39830.52,4354.458167,0.209049,0.450922,0.669153,0.352008,0.434217,0.516395,0.452915,0.450752,0.489920,0.0,1.0
17996,2022-01-21 03:00:00,39835.99,39944.45,38220.00,38465.65,9508.823455,0.132732,0.406045,0.661804,0.338314,0.517145,0.415356,0.482901,0.421536,0.466298,1.0,0.0
17997,2022-01-21 04:00:00,38465.65,39070.00,38437.15,38842.03,3091.420650,0.196624,0.379660,0.664193,0.363926,0.415324,0.539112,0.429001,0.484356,0.491540,1.0,0.0


In [10]:
df['UP'].value_counts()

1.0    9186
0.0    8780
Name: UP, dtype: int64

In [11]:
df['DOWN'].value_counts()

0.0    9189
1.0    8777
Name: DOWN, dtype: int64

看起來數據蠻平衡的

# 儲存資料

In [12]:
df.to_csv(file_name, index=False)

# 分割成X、y

In [13]:

X,y = list(),list()
ref_bar = 10

for i in range(len(df)-ref_bar):
    #df.iloc.values 會回傳一個numpy array
    X.append(df.iloc[i:i+ref_bar, 6:15].values) # i to i+ref_bar-1
    y.append(df.iloc[i+ref_bar-1, 15:].values) # i+ref_bar-1



In [14]:
X[0]#這裡會出現10個array，每個array裡面有9個數字，分別是RSI,MACD,OBV,CCI,OPEN_percent,CLOSE_percent,HIGH_percent,LOW_percent,VOLUME_percent

array([[0.43913889, 0.55133701, 0.12608327, 0.46256588, 0.52881702,
        0.51462743, 0.52018715, 0.48049549, 0.48945974],
       [0.43867025, 0.55161662, 0.12496563, 0.49576539, 0.51536909,
        0.51155285, 0.52433208, 0.47821686, 0.48939349],
       [0.38335495, 0.551469  , 0.1243774 , 0.46591742, 0.5118732 ,
        0.5029779 , 0.50973939, 0.46944876, 0.48923002],
       [0.44045613, 0.55174959, 0.12499103, 0.48810118, 0.50329239,
        0.51890279, 0.51308937, 0.47332878, 0.48954191],
       [0.37667693, 0.55150544, 0.1237806 , 0.42534265, 0.51951603,
        0.50097207, 0.51598351, 0.46473758, 0.4889751 ],
       [0.39051836, 0.55144598, 0.12451964, 0.45081074, 0.50112199,
        0.51337261, 0.51254345, 0.47740044, 0.48943827],
       [0.3796175 , 0.55136194, 0.12385315, 0.4362722 , 0.51372472,
        0.50980846, 0.51317905, 0.47269111, 0.48935657],
       [0.24120493, 0.5498691 , 0.12106759, 0.13851629, 0.50993134,
        0.48063427, 0.51314925, 0.44606877, 0.48657753],


In [15]:
y[0]

array([1.0, 0.0], dtype=object)

In [16]:
X = np.array(X)
X = torch.tensor(X, dtype=torch.float32)

In [17]:
y = np.array(y,dtype=np.float32)
y = torch.tensor(y, dtype=torch.float32)

In [18]:
y

tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        ...,
        [0., 1.],
        [1., 0.],
        [1., 0.]])

In [19]:
print(f"X shape: {X.shape} , y shape: {y.shape}")

X shape: torch.Size([17956, 10, 9]) , y shape: torch.Size([17956, 2])


In [20]:
X[0]

tensor([[0.4391, 0.5513, 0.1261, 0.4626, 0.5288, 0.5146, 0.5202, 0.4805, 0.4895],
        [0.4387, 0.5516, 0.1250, 0.4958, 0.5154, 0.5116, 0.5243, 0.4782, 0.4894],
        [0.3834, 0.5515, 0.1244, 0.4659, 0.5119, 0.5030, 0.5097, 0.4694, 0.4892],
        [0.4405, 0.5517, 0.1250, 0.4881, 0.5033, 0.5189, 0.5131, 0.4733, 0.4895],
        [0.3767, 0.5515, 0.1238, 0.4253, 0.5195, 0.5010, 0.5160, 0.4647, 0.4890],
        [0.3905, 0.5514, 0.1245, 0.4508, 0.5011, 0.5134, 0.5125, 0.4774, 0.4894],
        [0.3796, 0.5514, 0.1239, 0.4363, 0.5137, 0.5098, 0.5132, 0.4727, 0.4894],
        [0.2412, 0.5499, 0.1211, 0.1385, 0.5099, 0.4806, 0.5131, 0.4461, 0.4866],
        [0.1583, 0.5472, 0.1163, 0.1458, 0.4812, 0.4788, 0.4639, 0.4470, 0.4843],
        [0.1697, 0.5453, 0.1175, 0.2727, 0.4792, 0.5132, 0.4803, 0.4804, 0.4895]])

In [21]:
y[0]

tensor([1., 0.])

# 建立資料集類別

- 要繼承torch.utils.data.Dataset
- 要實作`__len__`、`__getitem__`
- 後面要用DataLoader取用

In [22]:
#用sklearn來分成train val test
# train:val:test = 6:3:1
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.75, random_state=42)

In [23]:
print(f"X_train shape: {X_train.shape} , y_train shape: {y_train.shape}")

X_train shape: torch.Size([10773, 10, 9]) , y_train shape: torch.Size([10773, 2])


In [24]:
print(f"X_test shape: {X_test.shape} , y_test shape: {y_test.shape}")

X_test shape: torch.Size([5388, 10, 9]) , y_test shape: torch.Size([5388, 2])


In [25]:
print(f"X_val shape: {X_val.shape} , y_val shape: {y_val.shape}")

X_val shape: torch.Size([1795, 10, 9]) , y_val shape: torch.Size([1795, 2])


In [26]:
class TrainDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [27]:
class ValDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [28]:
class TestDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [29]:
train_dataset = TrainDataset(X_train, y_train)
val_dataset = ValDataset(X_val, y_val)
test_dataset = TestDataset(X_test, y_test)

# 建立模型

- 要繼承torch.nn.Module
- 可能要多建立不同的模型，到時候看結果再調整 -> 先讓數據流得通，再去看成績做調整。

- ver1 -> CNN+MLP
- ver2 -> CNN+LSTM+MLP
- ver3 -> CNN+GRU+MLP
- ver4 -> CNN+LSTM+GRU+MLP

In [30]:
class SelectItem(torch.nn.Module):#這是用來取出多個輸出其中一個的輸出，如果不用sequential的話，就可以不用這個
    def __init__(self, item_index):
        super(SelectItem, self).__init__()
        self._name = 'selectitem'
        self.item_index = item_index

    def forward(self, inputs):
        return inputs[self.item_index]

In [31]:
class crypto_classfier_ver1(nn.Module): #CNN+MLP
    def __init__(self):
        super(crypto_classfier_ver1, self).__init__()
        self.name = "CCV-1"
        self.spare_layer = nn.Sequential(
            torch.nn.Conv1d(10, 4, 3, stride=4, padding=1),
            torch.nn.ReLU())
        self.net = nn.Sequential(
            torch.nn.Linear(4, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 2),
            torch.nn.Softmax(dim=0))
        self.fusion = nn.Sequential(
            torch.nn.Linear(6, 16),
            torch.nn.ELU(),
            torch.nn.Linear(16, 2),
            torch.nn.Softmax(dim=0))

    def forward(self, x):
        x = self.spare_layer(x)
        x = x.T
        x1 = self.net(x[0])
        x2 = self.net(x[1])
        x3 = self.net(x[2])
        x = torch.cat((x1,x2,x3),0)
        x = self.fusion(x)
        return x

In [32]:
class crypto_classfier_ver2(nn.Module): #CNN+LSTM+MLP
    def __init__(self):
        super(crypto_classfier_ver2, self).__init__()
        self.name = "CCV-2"
        self.net = nn.Sequential(
            torch.nn.Conv1d(10, 20, 3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(20, 40, 3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(40, 1, 3, stride=1, padding=1),
            torch.nn.Linear(9,64),
            torch.nn.Linear(64,128),
            torch.nn.LSTM(128, 64, 25),
            SelectItem(0),
            SelectItem(0),
            torch.nn.Linear(64,2),
            torch.nn.Softmax(dim=0))
    def forward(self, x):
        x = self.net(x)
        return x

In [33]:
class crypto_classfier_ver3(nn.Module):#CNN+GRU+MLP
    def __init__(self):
        super(crypto_classfier_ver3, self).__init__()
        self.name = "CCV-3"
        self.net = nn.Sequential(
            torch.nn.Conv1d(10, 20, 3, stride=1, padding=1),
            torch.nn.ELU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(20, 40, 3, stride=1, padding=1),
            torch.nn.ELU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(40, 1, 3, stride=1, padding=1),
            torch.nn.Linear(9,64),
            torch.nn.Linear(64,128),
            torch.nn.GRU(128, 64, 25),
            SelectItem(0),
            SelectItem(0),
            torch.nn.Linear(64,2),
            torch.nn.ELU()
            )
    def forward(self, x):
        x = self.net(x)
        # print(f"=>{F.softmax(x, dim=0)}")
        x = F.softmax(x, dim=0)
        return x

In [34]:
class crypto_classfier_ver4(nn.Module):#CNN+LSTM+GRU+MLP
    def __init__(self):
        super(crypto_classfier_ver4, self).__init__()
        self.name = "CCV-4"
        self.batch_size = 1
        self.spare_layer = torch.nn.Conv1d(10, 2, 3, stride=3, padding=1)
        self.gru = nn.Sequential(
            torch.nn.Linear(3, 16),
            torch.nn.ELU(),
            torch.nn.Linear(16, 128),
            torch.nn.ELU(),
            torch.nn.GRU(128, 64, 25),
            SelectItem(0),
            torch.nn.Linear(64, 2))
        self.lstm = nn.Sequential(
            torch.nn.Linear(3, 16),
            torch.nn.ELU(),
            torch.nn.Linear(16, 128),
            torch.nn.ELU(),
            torch.nn.LSTM(128, 64, 25),
            SelectItem(0),
            torch.nn.Linear(64, 2))
        self.fusion = nn.Sequential(
            SelectItem(0),
            SelectItem(0),
            torch.nn.Linear(4, 16),
            torch.nn.ELU(),
            torch.nn.Linear(16, 2),
            torch.nn.Softmax(dim=0)
            )
    def forward(self, x):
        x = self.spare_layer(x)
        x1 , x2 = x[0], x[1]
        x1 = x1.view(1,1,3)
        x2 = x2.view(1,1,3)
        out1 = self.gru(x1)
        out2 = self.lstm(x2)
        x = torch.cat((out1,out2),2)
        x = self.fusion(x)
        return x


In [35]:
lab_tensor = train_dataset[0][0]
print(lab_tensor.shape)
print(lab_tensor)

torch.Size([10, 9])
tensor([[0.2092, 0.5178, 0.1727, 0.4007, 0.5123, 0.4674, 0.4951, 0.4368, 0.4837],
        [0.2568, 0.5178, 0.1775, 0.3880, 0.4674, 0.5270, 0.4810, 0.4542, 0.4918],
        [0.2627, 0.5183, 0.1799, 0.4119, 0.5274, 0.5135, 0.5292, 0.5004, 0.4895],
        [0.3035, 0.5199, 0.1816, 0.4255, 0.5136, 0.5245, 0.5259, 0.4711, 0.4901],
        [0.2876, 0.5210, 0.1802, 0.4313, 0.5251, 0.5021, 0.5110, 0.4795, 0.4890],
        [0.3418, 0.5232, 0.1819, 0.4947, 0.5025, 0.5286, 0.5296, 0.4805, 0.4904],
        [0.3860, 0.5262, 0.1845, 0.5713, 0.5293, 0.5263, 0.5403, 0.4846, 0.4906],
        [0.3913, 0.5290, 0.1865, 0.5621, 0.5264, 0.5134, 0.5109, 0.4744, 0.4895],
        [0.3731, 0.5310, 0.1842, 0.5789, 0.5138, 0.5037, 0.5306, 0.4798, 0.4888],
        [0.3455, 0.5321, 0.1826, 0.5364, 0.5040, 0.4993, 0.5073, 0.4638, 0.4887]])


In [36]:
lab_models = [crypto_classfier_ver1(), crypto_classfier_ver2(), crypto_classfier_ver3(), crypto_classfier_ver4()]
out_data = list(map(lambda x: x(lab_tensor), lab_models))

for t in out_data:
    print(t)

tensor([0.5458, 0.4542], grad_fn=<SoftmaxBackward0>)
tensor([0.4760, 0.5240], grad_fn=<SoftmaxBackward0>)
tensor([0.4858, 0.5142], grad_fn=<SoftmaxBackward0>)
tensor([0.4479, 0.5521], grad_fn=<SoftmaxBackward0>)


## Note

1) model
    - `__init__`: define [layers](https://pytorch.org/docs/stable/nn.html)
    - forward: forward pass -> compute prediction
2) loss and optimizer
    - lr: learning rate [default=0.001]
    - momentum: momentum for optimizer [default=0.9]
    - criterion: loss function [in torch.nn] eg.nn.BCELoss()
    - optimizer: optimizer [in torch.optim] eg.torch.optim.SGD()
        - eg. optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
3) training loop
    - forward pass: compute prediction and loss

        ```python
        output = model(data)
        loss = criterion(output, target)
        ```
        
    - backward pass: loss.backward()
    - update weights: optimizer.step()
    - zero the gradients: optimizer.zero_grad()

# TODO

- 寫訓練方法
    - using dataloader
        - batch and epoch
- 寫驗證方法
    - using model.eval()


In [37]:
test_dataloader_temp = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)
loss_f = nn.MSELoss()
test_model = crypto_classfier_ver1()
for ind,(x,y) in enumerate(test_dataloader_temp):
    if ind == 50:
        print(x.shape)
        print(y.shape)
        x = x[0]
        y = y
        out= test_model(x)
        out = torch.tensor([1.,0.]) if out[0] > out[1] else torch.tensor([0.,1.])
        loss_val = loss_f(out, y)
        print(loss_val)
        print(out.shape)
        print(out)
        print(y)
        break

torch.Size([1, 10, 9])
torch.Size([1, 2])
tensor(1.)
torch.Size([2])
tensor([1., 0.])
tensor([[0., 1.]])


C:\Users\李軒豪\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [38]:
#setting loss function and optimizer
loss_functions = [nn.MSELoss(), nn.BCELoss(),nn.CrossEntropyLoss()]
optimizers = [torch.optim.Adam, torch.optim.SGD]
models = [crypto_classfier_ver1(), crypto_classfier_ver2(), crypto_classfier_ver3(), crypto_classfier_ver4()]
lr = 0.0001
epochs = 30
batch_size = 1 #batch size 是一次讀取多少筆資料 我這邊只能設1 因為我們的資料是一筆一筆的
#training
for model in models:
    for loss_function in loss_functions:
        for optimizer in optimizers:
            print("model: ", model.name)
            print("loss_function: ", loss_function)
            print("optimizer: ", optimizer)
            print("lr: ", lr)
            print("epochs: ", epochs)
            print("batch_size: ", batch_size)
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            model.to(device)
            print("device: ", device)
            optimizer = optimizer(model.parameters(), lr=lr)
            train_loss = []
            test_loss = []
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            for epoch in range(epochs):
                for i, (x, y) in enumerate(train_loader):
                    x = x[0]
                    y = y
                    x = x.to(device)
                    y = y.to(device)
                    optimizer.zero_grad()
                    out = model(x)
                    loss = loss_function(out, y)
                    loss.backward()
                    optimizer.step()
                    train_loss.append(loss.item())
                    if i % 1000 == 0:
                        print("Train epoch: ", epoch, "i: ", i, "loss: ", loss.item(), "out: ", out, "y: ", y)
                for i, (x, y) in enumerate(test_loader):
                    x = x[0]
                    y = y
                    x = x.to(device)
                    y = y.to(device)
                    out = model(x)
                    loss = loss_function(out, y)
                    test_loss.append([loss.item(), model.parameters()])
                    if i % 1000 == 0:
                        print("epoch: ", epoch, "i: ", i, "loss: ", loss.item())
                if epoch % 10 == 0:
                    print("epoch: ", epoch, "train_loss: ", train_loss[-1], "test_loss: ", test_loss[-1])



model:  CCV-1
loss_function:  MSELoss()
optimizer:  <class 'torch.optim.adam.Adam'>
lr:  0.0001
epochs:  30
batch_size:  1
device:  cpu
Train epoch:  0 i:  0 loss:  0.39143332839012146 out:  tensor([0.6256, 0.3744], grad_fn=<SoftmaxBackward0>) y:  tensor([[0., 1.]])
Train epoch:  0 i:  100 loss:  0.15367677807807922 out:  tensor([0.6080, 0.3920], grad_fn=<SoftmaxBackward0>) y:  tensor([[1., 0.]])
Train epoch:  0 i:  200 loss:  0.3598342835903168 out:  tensor([0.5999, 0.4001], grad_fn=<SoftmaxBackward0>) y:  tensor([[0., 1.]])
Train epoch:  0 i:  300 loss:  0.16996760666370392 out:  tensor([0.5877, 0.4123], grad_fn=<SoftmaxBackward0>) y:  tensor([[1., 0.]])
Train epoch:  0 i:  400 loss:  0.3334299623966217 out:  tensor([0.5774, 0.4226], grad_fn=<SoftmaxBackward0>) y:  tensor([[0., 1.]])
Train epoch:  0 i:  500 loss:  0.1873781979084015 out:  tensor([0.5671, 0.4329], grad_fn=<SoftmaxBackward0>) y:  tensor([[1., 0.]])
Train epoch:  0 i:  600 loss:  0.20298603177070618 out:  tensor([0.5495

KeyboardInterrupt: 